# Move Prediction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Xmaster6y/lczerolens/blob/main/docs/source/notebooks/features/move-prediction.ipynb)

## Setup

In [ ]:
MODE = "local"  # "colab" | "colab-dev" | "local"

: 

In [ ]:
if MODE == "colab":
    !pip install -q lczerolens[hf]
elif MODE == "colab-dev":
    !rm -r lczerolens
    !git clone https://github.com/Xmaster6y/lczerolens -b main
    !pip install -q ./lczerolens --extra hf

## Load a Model

Load a leela network from file (already converted to `onnx`):

In [ ]:
from lczerolens import LczeroModel

model = LczeroModel.from_hf("lczerolens/maia-1900")

To convert original weights see the section [Convert Official Weights](#convert-official-weights).

## Predict a Move

The defined model natively integrates with `python-chess`. Use the utils to predict a policy vector and obtain an UCI move:

In [ ]:
from lczerolens import LczeroBoard

# Get the model prediction
board = LczeroBoard()
out = model(board)
policy = out["policy"]

# Use the prediction
best_move_index = policy.argmax()
move = board.decode_move(best_move_index)

board.push(move)
display(board)

As most network are trained only on legal moves the argmax should only be computed on them:

In [ ]:
out = model(board)
policy = out["policy"]

legal_indices = board.get_legal_indices()
legal_policy = policy[0].gather(0, legal_indices)
best_move_index = legal_indices[legal_policy.argmax()]
move = board.decode_move(best_move_index)

board.push(move)
display(board)

To simplify this process, you can use the `Sampler` class as shown below:

In [ ]:
from lczerolens.sampling import PolicySampler

sampler = PolicySampler(model, use_argmax=True)
move, _ = next(iter(sampler.get_next_moves([board])))

board.push(move)
display(board)